In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import importlib as imp

from collections import namedtuple
from random import sample, shuffle
from functools import reduce
from itertools import accumulate
from math import floor, ceil, sqrt, log, pi
from matplotlib import pyplot as plt
from tensorflow.keras import layers, utils, losses, models as mds, optimizers

if imp.util.find_spec('aggdraw'): import aggdraw
if imp.util.find_spec('tensorflow_addons'): from tensorflow_addons import layers as tfa_layers
if imp.util.find_spec('tensorflow_models'): from official.vision.beta.ops import augment as visaugment
if imp.util.find_spec('tensorflow_probability'): from tensorflow_probability import distributions as tfd

2022-03-28 14:59:02.181001: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-28 14:59:02.181083: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [19]:
def mean_per_class_acc(y_true, y_pred):
    score, up_opt = tf.compat.v1.metrics.mean_per_class_accuracy(y_true, y_pred, num_classes=2)
    tf.print(score.eval())

    score = tf.identity(score)       
    return score

# metric = tf.compat.v1.metrics.mean_per_class_accuracy([1, 0], [[.2, .8], [.1, .9]], 2)
with tf.compat.v1.Session() as sess:
    y_true = tf.one_hot([1, 0], 2)
    y_pred = tf.constant([[.2, .8], [.1, .9]], shape=(2,2))
    score = mean_per_class_acc(y_true, y_pred)
    print(score)

FailedPreconditionError: Graph execution error:

Detected at node 'mean_accuracy_11/div_no_nan/ReadVariableOp' defined at (most recent call last):
    File "/home/broxoli/miniconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/broxoli/miniconda3/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/broxoli/miniconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/broxoli/miniconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/broxoli/miniconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/home/broxoli/miniconda3/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/home/broxoli/minicon